In [1]:
import os
import csv
import copy
import math

In [2]:
graphs_dir = "graphs"
results_dir = "results"

In [3]:
graph_name = "graph_3_nodes"

graph = {}
with open(os.path.join(graphs_dir, f"{graph_name}.txt"), "r") as f:
    line = f.readline()
    while line:
        node_edges = line.split()
        node = node_edges[0]
        edges = node_edges[1:]
        # graph[node] = set(edges)
        graph[node] = edges
        line = f.readline()
graph

{'A': ['C', 'B'], 'B': ['A', 'C'], 'C': ['B', 'A']}

In [4]:
nodes = list(graph.keys())
node_positions = {v: i for i, v in enumerate(nodes)}

In [5]:
degree_of_nodes = {n: 2 for n in nodes}

print("Degree of all nodes (starting from 0):")
degree_of_nodes # start from 0

Degree of all nodes (starting from 0):


{'A': 2, 'B': 2, 'C': 2}

In [6]:
configurations = {
    tuple([0 for i in range(len(nodes))])
}
# perturb each state at a time for all states in configurations and accumulate the same in the configurations for next state to perturb
for n in nodes:
    node_pos = node_positions[n]
    config_copy = copy.deepcopy(configurations)
    for i in range(1, degree_of_nodes[n]+1):
        for cc in config_copy:
            cc = list(cc)
            cc[node_pos] = i
            configurations.add(tuple(cc))
            
print("All possible configurations:")
# configurations, len(configurations)

All possible configurations:


In [7]:
def check_if_only_one_eligible_process(state):
    """check invariant"""
    bottom = 0
    top = len(state) - 1
    eligible_nodes = 0
    for i, node_state in enumerate(state):
        if i == bottom:
            if (node_state + 1) % 3 == state[i + 1]:
                eligible_nodes += 1

        elif i == top:
            if state[i - 1] == state[0] and (state[i - 1] + 1) % 3 != node_state:
                eligible_nodes += 1
        
        else:
            if (node_state + 1) % 3 == state[i - 1]:
                eligible_nodes += 1
            
            if (node_state + 1) % 3 == state[i + 1]:
                eligible_nodes += 1

    return eligible_nodes == 1

In [8]:
invariants = set()
for state in configurations:
    # dijkstra specific
    if check_if_only_one_eligible_process(state):
        invariants.add(state)

print("Invariants and Count of Invariants:")
invariants, len(invariants)

Invariants and Count of Invariants:


({(0, 0, 0),
  (0, 0, 1),
  (0, 0, 2),
  (0, 1, 0),
  (0, 1, 1),
  (0, 2, 1),
  (0, 2, 2),
  (1, 0, 0),
  (1, 0, 2),
  (1, 1, 0),
  (1, 1, 1),
  (1, 1, 2),
  (1, 2, 1),
  (1, 2, 2),
  (2, 0, 0),
  (2, 0, 2),
  (2, 1, 0),
  (2, 1, 1),
  (2, 2, 0),
  (2, 2, 1),
  (2, 2, 2)},
 21)

In [9]:
program_transitions_rank = {}
for inv in invariants:
    program_transitions_rank[inv] = {"L": 0, "C": 1, "A": 0, "Ar": 0, "M": 0}

print("Program transitions rank for invariants:")
program_transitions_rank

Program transitions rank for invariants:


{(2, 0, 2): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (0, 1, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 2, 2): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 1, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (1, 2, 2): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (0, 0, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (0, 2, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (1, 1, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 2, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (1, 2, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (0, 0, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (1, 1, 2): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (1, 0, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 0, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 2, 0): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (0, 1, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (2, 1, 1): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (0, 0, 2): {'L': 0, 'C': 1, 'A': 0, 'Ar': 0, 'M': 0},
 (0, 2, 2)

In [10]:
def is_program_transition(perturb_pos, start_state, dest_state):
    # dijkstra specific

    if start_state in invariants and dest_state in invariants:
        return False

    s_prev = start_state[perturb_pos]
    s_new = dest_state[perturb_pos]

    node = nodes[perturb_pos]

    neighbor_pos = [node_positions[n] for n in graph[node]]
    neighbor_states = [start_state[i] for i in neighbor_pos]
    left_state, right_state = neighbor_states

    if node == nodes[0]:  # bottom
        return (s_prev + 1) % 3 == right_state and s_new == (s_prev - 1) % 3

    elif node == nodes[-1]:  # top
        return (
            left_state == right_state
            and (left_state + 1) % 3 != s_prev
            and s_new == (left_state + 1) % 3
        )

    else:  # others
        if (s_prev + 1) % 3 == left_state:
            return s_new == left_state
        elif (s_prev + 1) % 3 == right_state:
            return s_new == right_state
    return False


# is_program_transition(0, [2, 0, 1])

In [11]:
def get_program_transitions(start_state):
    # dijkstra specific
    program_transitions = set()
    for position, _ in enumerate(start_state):
        possible_node_colors = set(range(degree_of_nodes[nodes[position]]+1))
        for perturb_val in possible_node_colors:
            perturb_state = list(start_state)
            perturb_state[position] = perturb_val
            perturb_state = tuple(perturb_state)
            if perturb_state != start_state:
                if is_program_transition(position, start_state, perturb_state):
                    program_transitions.add(perturb_state)
                # else:
                #     if start_state in program_transitions_rank:
                #         cvfs_in.add(perturb_state)
                #     else:
                #         cvfs_out.add(perturb_state)
    return {"program_transitions": program_transitions}

In [12]:
def get_cvfs(start_state):
    cvfs_in = set()
    cvfs_out = set()
    for position, _ in enumerate(start_state):
        possible_node_colors = set(range(degree_of_nodes[nodes[position]]+1))
        for perturb_val in possible_node_colors:
            perturb_state = list(start_state)
            perturb_state[position] = perturb_val
            perturb_state = tuple(perturb_state)
            if perturb_state != start_state:
                if start_state in invariants:
                    cvfs_in.add(perturb_state)
                else:
                    cvfs_out.add(perturb_state)
    return {"cvfs_in": cvfs_in, "cvfs_out": cvfs_out}


In [13]:
program_transitions_n_cvf = {}

for state in configurations:
    program_transitions_n_cvf[state] = {**get_program_transitions(state), **get_cvfs(state)}

In [14]:
program_transitions_n_cvf

{(2, 0, 2): {'program_transitions': set(),
  'cvfs_in': {(0, 0, 2),
   (1, 0, 2),
   (2, 0, 0),
   (2, 0, 1),
   (2, 1, 2),
   (2, 2, 2)},
  'cvfs_out': set()},
 (0, 1, 0): {'program_transitions': set(),
  'cvfs_in': {(0, 0, 0),
   (0, 1, 1),
   (0, 1, 2),
   (0, 2, 0),
   (1, 1, 0),
   (2, 1, 0)},
  'cvfs_out': set()},
 (2, 2, 2): {'program_transitions': set(),
  'cvfs_in': {(0, 2, 2),
   (1, 2, 2),
   (2, 0, 2),
   (2, 1, 2),
   (2, 2, 0),
   (2, 2, 1)},
  'cvfs_out': set()},
 (2, 1, 0): {'program_transitions': set(),
  'cvfs_in': {(0, 1, 0),
   (1, 1, 0),
   (2, 0, 0),
   (2, 1, 1),
   (2, 1, 2),
   (2, 2, 0)},
  'cvfs_out': set()},
 (1, 2, 2): {'program_transitions': set(),
  'cvfs_in': {(0, 2, 2),
   (1, 0, 2),
   (1, 1, 2),
   (1, 2, 0),
   (1, 2, 1),
   (2, 2, 2)},
  'cvfs_out': set()},
 (0, 0, 1): {'program_transitions': set(),
  'cvfs_in': {(0, 0, 0),
   (0, 0, 2),
   (0, 1, 1),
   (0, 2, 1),
   (1, 0, 1),
   (2, 0, 1)},
  'cvfs_out': set()},
 (0, 2, 1): {'program_transitions'

In [15]:
unranked_states = set(program_transitions_n_cvf.keys()) - set(program_transitions_rank.keys())

In [16]:
# rank the states that has all the paths to the ranked one
while unranked_states:
    ranked_states = set(program_transitions_rank.keys())
    remove_from_unranked_states = set()
    for state in unranked_states:
        dests = program_transitions_n_cvf[state]['program_transitions']
        if dests - ranked_states:       # some desitnations states are yet to be ranked
            pass
        else:                           # all the destination has been ranked
            total_path_length = 0
            path_count = 0
            _max = 0
            for succ in dests:
                path_count += program_transitions_rank[succ]["C"]
                total_path_length += program_transitions_rank[succ]["L"] + program_transitions_rank[succ]["C"]
                _max = max(_max, program_transitions_rank[succ]["M"])
            program_transitions_rank[state] = {
                "L": total_path_length,
                "C": path_count,
                "A": total_path_length/path_count,
                "Ar": math.ceil(total_path_length/path_count),
                "M": _max + 1
            }
            remove_from_unranked_states.add(state)
    unranked_states -= remove_from_unranked_states

In [17]:
pt_rank_effect = {}

for state, pt_cvfs in program_transitions_n_cvf.items():
    for pt in pt_cvfs['program_transitions']:
        pt_rank_effect[(state, pt)] = {
            "Ar": program_transitions_rank[pt]["Ar"] - program_transitions_rank[state]["Ar"],
            "M": program_transitions_rank[pt]["M"] - program_transitions_rank[state]["M"]
        }

In [18]:
cvfs_in_rank_effect = {}
cvfs_out_rank_effect = {}

for state, pt_cvfs in program_transitions_n_cvf.items():
    for cvf in pt_cvfs['cvfs_in']:
        cvfs_in_rank_effect[(state, cvf)] = {
            "Ar": program_transitions_rank[cvf]["Ar"] - program_transitions_rank[state]["Ar"],
            "M": program_transitions_rank[cvf]["M"] - program_transitions_rank[state]["M"]
        }
    for cvf in pt_cvfs['cvfs_out']:
        cvfs_out_rank_effect[(state, cvf)] = {
            "Ar": program_transitions_rank[cvf]["Ar"] - program_transitions_rank[state]["Ar"],
            "M": program_transitions_rank[cvf]["M"] - program_transitions_rank[state]["M"]
        }

In [19]:
import pandas as pd

In [20]:
pt_rank_effect_ = []
for state in pt_rank_effect:
    pt_rank_effect_.append({"state": state, **pt_rank_effect[state]})

pt_rank_effect_df = pd.DataFrame(pt_rank_effect_)

In [21]:
cvfs_in_rank_effect_ = []
for state in cvfs_in_rank_effect:
    cvfs_in_rank_effect_.append({"state": state, **cvfs_in_rank_effect[state]})
    
cvfs_in_rank_effect_df = pd.DataFrame(cvfs_in_rank_effect_)

In [22]:
cvfs_out_rank_effect_ = []
for state in cvfs_out_rank_effect:
    cvfs_out_rank_effect_.append({"state": state, **cvfs_out_rank_effect[state]})

cvfs_out_rank_effect_df = pd.DataFrame(cvfs_out_rank_effect_)

In [23]:
pt_avg_counts = pt_rank_effect_df['Ar'].value_counts()
pt_max_counts = pt_rank_effect_df['M'].value_counts()
cvf_in_avg_counts = cvfs_in_rank_effect_df['Ar'].value_counts()
cvf_in_max_counts = cvfs_in_rank_effect_df['M'].value_counts()
cvf_out_avg_counts = cvfs_out_rank_effect_df['Ar'].value_counts()
cvf_out_max_counts = cvfs_out_rank_effect_df['M'].value_counts()

In [24]:
cvf_out_avg_counts

Ar
-1    18
-2    15
 1     3
Name: count, dtype: int64

In [26]:
fieldnames = ["Rank Effect", "PT (Max)", "PT (Avg)", "CVF In (Max)", "CVF In (Avg)", "CVF Out (Max)", "CVF Out (Avg)"]
with open(os.path.join(results_dir, f"rank_effect_{graph_name}.csv"), "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for re in sorted(
        set(pt_avg_counts.index) |
        set(pt_max_counts.index) |
        set(cvf_in_avg_counts.index) |
        set(cvf_in_max_counts.index) |
        set(cvf_out_avg_counts.index) |
        set(cvf_out_max_counts.index)
    ):
        writer.writerow({
            "Rank Effect": re,
            "PT (Max)": pt_max_counts.get(re, 0),
            "PT (Avg)": pt_avg_counts.get(re, 0),
            "CVF In (Max)": cvf_in_max_counts.get(re, 0),
            "CVF In (Avg)": cvf_in_avg_counts.get(re, 0),
            "CVF Out (Max)": cvf_out_max_counts.get(re, 0),
            "CVF Out (Avg)": cvf_out_avg_counts.get(re, 0),
        })

In [27]:
pt_rank_ = []
for state in program_transitions_rank:
    pt_rank_.append({"state": state, **program_transitions_rank[state]})

pt_rank_df = pd.DataFrame(pt_rank_)

In [28]:
pt_avg_counts = pt_rank_df['Ar'].value_counts()
pt_max_counts = pt_rank_df['M'].value_counts()

In [29]:
fieldnames = ["Rank", "Count (Max)", "Count (Avg)"]
with open(os.path.join(results_dir, f"rank_{graph_name}.csv"), "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for rank in sorted(set(pt_avg_counts.index)|set(pt_max_counts.index)):
        writer.writerow({"Rank": rank, "Count (Max)": pt_max_counts.get(rank, 0), "Count (Avg)": pt_avg_counts.get(rank, 0)})